In [1]:

# coding: utf-8


# dependencies
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
import time
import data_utils
import matplotlib.pyplot as plt

In [2]:
X, Y, en_word2idx, en_idx2word, en_vocab, zh_word2idx, zh_idx2word, zh_vocab = data_utils.read_dataset('en_n_zh.pkl')
print(en_vocab)

['you', '-', 'I', 'to', 'the', 's', 'a', 't', 'of', 'it', 'me', 'is', 'and', "I'", 'your', 'You', 'that', 'in', 'are', 'this', 'be', 'on', 're', 'What', 'for', 'have', 'my', 'm', "don'", 'here', 'know', 'not', 'do', 'il', 'go', 'we', 'no', 'what', 'think', 'all', 'one', 'with', 'Harry', 'can', 'was', 'he', 'just', 'up', 'get', 'so', 'but', 'school', 'like', "it'", 'him', 'The', 've', 'about', "It'", 'right', 'out', 'come', 'at', "You'", 'Oh', 'her', 'will', 'Dobby', 'Big', 'if', "you'", 'brother', 'Why', 'been', 'back', 'now', 'see', 'how', 'How', 'by', 'from', 'them', 'did', 'sir', 'time', 'If', 'there', 'No', 'who', 'And', 'Potter', "can'", 'only', 'tell', 'Well', '--', 'our', 'going', 'too', 'they', 'Yes', 'This', 'Come', 'has', 'were', 'We', 'But', 'want', 'take', 'even', 'Do', 'never', 'really', 'look', "didn'", 'Hey', 'day', "That'", 'way', 'Are', 'when', 'am', 'well', "What'", 'us', "Don'", 'little', 'two', 'home', 'told', 'must', 'So', 'years', "we'", 'his', 'It', 'said', 'say'

In [3]:
# data processing
def replace_sentence_with_unk(sentence,en_vocab):
    for x in sentence:
        for y in range(len(x)):
            if x[y]  not in en_vocab:
                x[y]='<ukn>'
# data padding
def data_padding(x, y=None, length = 16):
    for i in range(len(x)):
        #x[i] = x[i] + (length - len(x[i])) * [en_word2idx['<pad>']]
        #y[i] = [zh_word2idx['<go>']] + y[i] + [zh_word2idx['<eos>']] + (length-len(y[i])) * [zh_word2idx['<pad>']]
        x[i] = x[i] + (length - len(x[i])) * [en_word2idx['<pad>']]
        if y is not None:
            y[i] = y[i] + (length - len(y[i])) * [zh_word2idx['<pad>']]
import random
def generate_useless_sentence(X,vocab):
    useless_data=[]
    for i in range(len(X)):
        sentence_length=len(X[random.randint(0,len(X)-1)])
        temp_sentence=[random.randint(2,len(vocab)) for x in range(sentence_length)]
        useless_data.append(temp_sentence)
    return useless_data
def mix_data(source_sentences,useless_sentences):
    over_all_data=[]
    label_of_mix_data=[]
    for x in range(len(source_sentences)+len(useless_sentences)):
        if(random.randint(0,1) is 0):
            if len(source_sentences) is 0:
                continue
            over_all_data.append(source_sentences[0])
            label_of_mix_data.append(1)
            del source_sentences[0]
        else:
            if len(useless_sentences) is 0:
                continue
            over_all_data.append(useless_sentences[0])
            label_of_mix_data.append(0)
            del useless_sentences[0]
    return over_all_data,label_of_mix_data
#data_padding(X, Y)
#print(Y)
def process_data(X,en_vocab,Y,zh_vocab):
    en_useless=generate_useless_sentence(X,en_vocab)
    zh_useless=generate_useless_sentence(Y,zh_vocab)
    en_all,en_label=mix_data(X,en_useless)
    zh_all,zh_label=mix_data(Y,zh_useless)
    data_padding(en_all)
    data_padding(zh_all)
    return en_all,en_label,zh_all,zh_label
en_all,en_label,zh_all,zh_label=process_data(X,en_vocab,Y,zh_vocab)
en_all_train,  en_all_test, en_label_train, en_label_test = train_test_split(en_all, en_label, test_size = 0.1)

In [4]:
n_inputs = 16  # MNIST
n_hidden0 = 32
n_hidden1 = 64
n_hidden2 = 32
#n_hidden3 = 64
#n_hidden4 = 32
n_outputs = 2

In [5]:
#reset_graph()

X_var = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
Y_var = tf.placeholder(tf.int64, shape=(None), name="y") 

In [6]:
with tf.name_scope("dnn"):
    hidden0 = tf.layers.dense(X_var, n_hidden0, name="hidden0",
                              activation=tf.nn.relu)
    hidden1 = tf.layers.dense(hidden0, n_hidden1, name="hidden1",
                              activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2",
                              activation=tf.nn.relu)
    #hidden3 = tf.layers.dense(hidden2,n_hidden3,name="hidden3",
                            # activation=tf.nn.relu)
    #hidden4 = tf.layers.dense(hidden3,n_hidden4,name='hidden4',
                            # activation=tf.nn.relu)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")

In [7]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=Y_var, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    softmax=tf.nn.softmax(logits=logits)
    softmax_mean=tf.reduce_mean(tf.slice(softmax,[0,1],[-1,1]))

In [8]:
learning_rate = 0.0001

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [9]:
#X, Y, en_word2idx, en_idx2word, en_vocab, zh_word2idx, zh_idx2word, zh_vocab 

In [10]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, Y_var, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [11]:
#print(X)
#print([zh_idx2word[id] for id in range(len(zh_vocab))])


In [12]:
#useless_data=generate_useless_sentence(X,zh_word2idx,zh_idx2word,zh_vocab)

In [13]:
#[zh_idx2word[id] for id in useless_data[0]]
#print(len(useless_data[0]))

In [14]:
#[random.randint(0,1) for x in range(100)]

In [15]:
n_epochs = 1000001
batch_size = 50
n_batches = int(np.ceil(len(en_all) / batch_size))
init = tf.global_variables_initializer()

In [16]:
def next_batch(en_all,en_vocab,batch_size):
    current_position=0
    while 1:
        if current_position <= len(en_all):
            yield en_all[current_position:current_position+batch_size],en_label[current_position:\
                                                                                current_position+batch_size]
        else:
            current_position=0
            yield en_all[current_position:current_position+batch_size],en_label[current_position:\
                                                                               current_position+batch_size]
        current_position+=batch_size

In [17]:
get=next_batch(en_all,en_vocab,batch_size)
print(len(en_all_train))
print(batch_size)
checkpoint_path = "./data/check_sentence.ckpt"
saver = tf.train.Saver()

1967
50


In [18]:

n_epochs = 1001
batch_size = 50
n_batches = int(np.ceil(len(en_all_train) / batch_size))
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        for iteration in range(len(en_all_train) // batch_size):
            X_batch, Y_batch = next(get)
            #X_batch=tf.convert_to_tensor(X_batch)
            #Y_batch=tf.convert_to_tensor(Y_batch)
            #print(X_batch)
            sess.run(training_op, feed_dict={X_var: X_batch, Y_var: Y_batch})
            #print(1)
        accuracy_val \
        = sess.run(accuracy, feed_dict={X_var: en_all_test, Y_var: en_label_test})
        if epoch % 10 is 0:
            print(accuracy_val)
    
    
    saver.save(sess, checkpoint_path)


0.5753425
0.74429226
0.7716895
0.82191783
0.80365294
0.8082192
0.826484
0.8310502
0.8310502
0.8310502
0.84474885
0.8584475
0.84931505
0.8630137
0.86757994
0.8630137
0.8584475
0.80365294
0.8995434
0.89041096
0.89041096
0.89497715
0.7853881
0.89497715
0.84474885
0.9041096
0.913242
0.9041096
0.88584477
0.89497715
0.8995434
0.89497715
0.9041096
0.9041096
0.9041096
0.913242
0.9041096
0.89041096
0.88584477
0.8767123
0.913242
0.91780823
0.91780823
0.913242
0.826484
0.9041096
0.8995434
0.91780823
0.9086758
0.913242
0.91780823
0.913242
0.9086758
0.913242
0.9086758
0.9269406
0.91780823
0.9223744
0.93607306
0.9269406
0.9223744
0.94063926
0.94063926
0.9269406
0.9223744
0.9269406
0.84931505
0.9269406
0.93607306
0.93607306
0.94063926
0.9315069
0.9269406
0.913242
0.94063926
0.9315069
0.9315069
0.94063926
0.9269406
0.94520545
0.9269406
0.94520545
0.93607306
0.94063926
0.94520545
0.93607306
0.93607306
0.9315069
0.84931505
0.9497717
0.9315069
0.93607306
0.94063926
0.9315069
0.9269406
0.9086758
0.9452054

In [21]:
sentences=[['用', '用', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾'], ['冊', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾'], ['冊', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾'], ['銷售', '銷售', '矛盾', '矛盾', '矛盾', '矛盾', '五', '五', '五', '五', '五', '五', '五', '五', '五', '五', '五', '老實說'], ['冊', '冊', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾'], ['冊', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾'], ['冊', '矛盾', '矛盾', '矛盾', '以及', '以及', '以及', '以及', '以及', '以及', '以及', '以及', '以及', '以及', '以及', '以及', '以及', '以及'], ['圖書館', '圖書館', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '救', '救', '救', '救'], ['冊', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾'], ['站在', '銷售', '聽錯', '聽錯', '聽錯', '聽錯', '聽錯', '聽錯', '五', '五', '五', '五', '五', '五', '五', '五', '五', '五'], ['冊', '冊', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾'], ['冊', '冊', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾'], ['事情', '事情', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾'], ['冊', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾'], ['冊', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾'], ['冊', '冊', '矛盾', '以及', '以及', '以及', '以及', '以及', '以及', '以及', '以及', '以及', '以及', '以及', '以及', '行李車', '行李車', '行李車'], ['冊', '矛盾', '矛盾', '矛盾', '以及', '以及', '以及', '以及', '以及', '以及', '以及', '以及', '以及', '轟', '轟', '轟', '轟', '轟'], ['站在', '用', '行李車', '行李車', '行李車', '行李車', '行李車', '行李車', '行李車', '行李車', '行李車', '行李車', '行李車', '行李車', '行李車', '行李車', '行李車', '行李車'], ['銷售', '銷售', '比的', '矛盾', '矛盾', '矛盾', '矛盾', '感到', '感到', '感到', '感到', '救', '救', '救', '救', '救', '救', '救'], ['冊', '冊', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '轟', '轟', '轟'], ['銷售', '銷售', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾'], ['冊', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾'], ['冊', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾'], ['冊', '冊', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾'], ['銷售', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾'], ['冊', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾'], ['銷售', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾'], ['冊', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾'], ['冊', '冊', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾'], ['聽錯', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾'], ['冊', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾'], ['冊', '時代', '矛盾', '矛盾', '以及', '以及', '以及', '以及', '以及', '以及', '以及', '轟', '轟', '轟', '轟', '轟', '轟', '轟'], ['冊', '冊', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾'], ['圖書館', '聽錯', '聽錯', '聽錯', '聽錯', '聽錯', '聽錯', '聽錯', '行李車', '聽錯', '行李車', '行李車', '行李車', '行李車', '行李車', '聽錯', '聽錯', '聽錯'], ['冊', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾'], ['想像', '小姐', '小姐', '時代', '時代', '矛盾', '矛盾', '矛盾', '矛盾', '救', '救', '救', '救', '救', '救', '救', '救', '救'], ['銷售', '銷售', '躺在', '躺在', '躺在', '躺在', '躺在', '躺在', '躺在', '躺在', '躺在', '躺在', '躺在', '躺在', '躺在', '躺在', '躺在', '躺在'], ['想像', '想像', '用', '用', '用', '聽錯', '聽錯', '聽錯', '聽錯', '犯', '犯', '犯', '觸', '觸', '觸', '觸', '觸', '觸'], ['冊', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾'], ['站在', '站在', '矛盾', '矛盾', '矛盾', '矛盾', '冷血', '冷血', '冷血', '冷血', '冷血', '冷血', '冷血', '冷血', '冷血', '冷血', '下次', '下次'], ['冊', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾'], ['冊', '聽錯', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾'], ['冊', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾'], ['冊', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾'], ['冊', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾'], ['圖書館', '冊', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '行李車', '行李車', '行李車', '行李車', '行李車', '行李車', '行李車', '行李車', '行李車', '行李車', '行李車'], ['用', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾'], ['冊', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾'], ['冊', '機', '機', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '轟', '轟', '轟', '轟', '轟', '轟', '轟', '轟', '轟', '轟'], ['聽錯', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '五', '五', '五', '五', '五', '五', '五', '五', '轟', '轟'], ['銷售', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾'], ['圖書館', '圖書館', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '聽錯', '聽錯', '聽錯', '聽錯', '聽錯', '聽錯', '聽錯', '聽錯', '聽錯'], ['冊', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾'], ['待在', '待在', '症', '症', '不憤', '不憤', '會長', '會長', '會長', '會長', '會長', '會長', '會長', '會長', '說', '說', '說', '說'], ['冊', '冊', '捆', '捆', '捆', '行李車', '行李車', '行李車', '行李車', '行李車', '行李車', '躺在', '行李車', '行李車', '行李車', '兩個', '兩個', '兩個'], ['冊', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾'], ['冊', '冊', '聽錯', '聽錯', '聽錯', '聽錯', '聽錯', '聽錯', '聽錯', '聽錯', '聽錯', '聽錯', '聽錯', '聽錯', '聽錯', '聽錯', '聽錯', '聽錯'], ['站在', '聽錯', '矛盾', '矛盾', '矛盾', '以及', '以及', '以及', '以及', '以及', '以及', '以及', '以及', '以及', '以及', '以及', '以及', '以及'], ['站在', '借光', '捆', '矛盾', '矛盾', '犯', '犯', '犯', '犯', '犯', '犯', '犯', '犯', '犯', '犯', '犯', '犯', '犯'], ['站在', '站在', '觸', '觸', '觸', '觸', '觸', '觸', '觸', '觸', '觸', '觸', '觸', '觸', '觸', '觸', '觸', '觸'], ['冊', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾'], ['站在', '冊', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '冷血', '冷血', '冷血', '冷血', '冷血', '冷血', '冷血', '冷血', '冷血'], ['站在', '冊', '時代', '時代', '以及', '以及', '以及', '以及', '以及', '以及', '老實說', '老實說', '老實說', '老實說', '老實說', '老實說', '老實說', '老實說'], ['銷售', '銷售', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾', '矛盾']]

In [24]:
print(zh_word2idx['矛盾'])

840


In [23]:
check_sentence(sentences,zh_vocab)

INFO:tensorflow:Restoring parameters from ./data/check_sentence.ckpt
accuracy.{} 0.9315069


UnboundLocalError: local variable 'sentence' referenced before assignment

In [20]:
op=tf.reduce_sum(tf.slice([[1,3],[1,3]],[0,1],[-1,1]))
with tf.Session() as sess:
    saver.restore(sess, checkpoint_path)
    print(sess.run(softmax_mean,feed_dict={X_var:en_all_test }))

INFO:tensorflow:Restoring parameters from ./data/check_sentence.ckpt
0.47703755
